In [1]:
import sys
sys.path.append('../')
%reload_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin

import utils.paramUtil as paramUtil
from options.train_options import TrainCompOptions
from utils.plot_script import *

from datasets import Text2MotionDataset

from mmcv.runner import get_dist_info, init_dist
from mmcv.parallel import MMDistributedDataParallel, MMDataParallel
import torch
import torch.distributed as dist
from mylib import animate3d, kit_bone, kit_kit_bone, visualize_2motions
from utils.motion_process import recover_from_ric
from datasets import build_dataloader

import torch
import torch.nn.functional as F
from mask_model.motiontransformer import MotionTransformerOnly, generate_src_mask
from mask_model.quantize import VectorQuantizer2
from mask_model.util import hinge_d_loss, vanilla_d_loss, MeanMask

In [2]:
class Temp:
    def __init__(self):
        print('mock:: opt')
opt = Temp()


# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/train_options.py
opt.num_layers = 8
opt.latent_dim = 512
opt.diffusion_steps = 1000
opt.no_clip = False
opt.no_eff = False
opt.num_epochs = 50
opt.lr = 2e-4
# opt.batch_size = 32
opt.times = 1
opt.feat_bias = 1
opt.is_continue = False
opt.log_every = 50
opt.save_every_e = 5
opt.log_every = 5
opt.eval_every_e = 5
opt.save_latest = 500
opt.is_train = True

# mock from cmd
opt.name = 'TEMP'
opt.batch_size = 2 # [INFO] modified
opt.times = 50
opt.num_epochs = 1 # [INFO] modified
opt.dataset_name = 'kit' # [INFO] or t2m or kit
opt.num_layers = 8
opt.diffusion_steps = 1000
opt.data_parallel = True
opt.gpu_id = [0,1,2,3]
opt.device = torch.device("cuda")

# mock from /home/epinyoan/git/MotionDiffuse/text2motion/options/base_options.py
opt.checkpoints_dir = '../checkpoints'
opt.distributed = False

opt.save_root = pjoin(opt.checkpoints_dir, opt.dataset_name, opt.name)
opt.model_dir = pjoin(opt.save_root, 'model')
opt.meta_dir = pjoin(opt.save_root, 'meta')

if opt.dataset_name == 't2m':
    opt.data_root = '../data/HumanML3D' # change root to '../'
    opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
    opt.text_dir = pjoin(opt.data_root, 'texts')
    opt.joints_num = 22
    radius = 4
    fps = 20
    opt.max_motion_length = 196
    dim_pose = 263
    kinematic_chain = paramUtil.t2m_kinematic_chain
elif opt.dataset_name == 'kit':
    opt.data_root = '../data/KIT-ML' # change root to '../'
    opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
    opt.text_dir = pjoin(opt.data_root, 'texts')
    opt.joints_num = 21
    radius = 240 * 8
    fps = 12.5
    dim_pose = 251
    opt.max_motion_length = 196
    kinematic_chain = paramUtil.kit_kinematic_chain

opt.debug = True # [INFO] added
opt.corrupt = 'diffusion' # [INFO] added ['mask', 'diffusion']
dim_word = 300
mean = np.load(pjoin(opt.data_root, 'Mean.npy'))
std = np.load(pjoin(opt.data_root, 'Std.npy'))

train_split_file = pjoin(opt.data_root, 'train.txt')


train_dataset = Text2MotionDataset(opt, mean, std, train_split_file, opt.times)

mock:: opt


100%|██████████| 300/300 [00:00<00:00, 2645.90it/s]


### Vector Quantization (CodeBook)

In [4]:
vq = VectorQuantizer2(n_e = 1024,
                      e_dim = dim_pose)
# quant, emb_loss, info = vq(torch.rand([5, 256, 16, 16]))
z_q = vq(torch.rand([2, 196, dim_pose]))
z_q.shape

torch.Size([2, 196, 251])

### Loss
Something to look further
- length set to "MotionTransformer" ex. discriminator(recon, length=length)

In [5]:
train_loader = build_dataloader(
        train_dataset,
        samples_per_gpu=opt.batch_size,
        drop_last=True,
        workers_per_gpu=1,
        shuffle=True,
        dist=opt.distributed,
        num_gpus=1)
text, motions, length = next(iter(train_loader))
motions = motions.float()
src_mask = generate_src_mask(motions.shape[1], length).to(motions.device).unsqueeze(-1)
mean_mask = MeanMask(src_mask, dim_pose)

In [12]:
latent_dim = 256
encoder = MotionTransformerOnly(input_feats=dim_pose, latent_dim=latent_dim, num_layers=8)
decoder = MotionTransformerOnly(input_feats=latent_dim, output_feats=dim_pose, latent_dim=latent_dim, num_layers=8)
discriminator = MotionTransformerOnly(input_feats=dim_pose, output_feats=1, latent_dim=latent_dim, num_layers=4)
q_beta = 0.25
quantize = VectorQuantizer2(n_e = 1024,
                            e_dim = latent_dim)

z = encoder(motions, src_mask=src_mask, length=length)
z_q = quantize(z) * src_mask
qloss = mean_mask.mean((z_q.detach()-z)**2 * src_mask) + 0.25 * \
                   mean_mask.mean((z_q - z.detach()) ** 2 * src_mask)
z_q = z + (z_q - z).detach()
recon = decoder(z_q, src_mask=src_mask, length=length)
logits_fake = discriminator(recon, src_mask=src_mask, length=length)

z_q torch.Size([2, 196, 256])


In [7]:
diff = torch.abs(motions - recon)
a = (diff * src_mask).sum()/(src_mask.sum() * dim_pose)
# src_mask.sum(), src_mask.shape
mean_mask = MeanMask(src_mask, dim_pose)
mean_mask.mean(diff * src_mask), a


(tensor(0.8255, grad_fn=<DivBackward0>),
 tensor(0.8255, grad_fn=<DivBackward0>))

In [39]:
##### GAN loss
# [TODO] mask the shorter frames for gan loss
rec_loss = mean_mask.mean(torch.abs(motions - recon) * src_mask)
# [TODO] clarify: discriminator of VQGAN output only 1x30x30 from input 3x256x256
g_loss = -mean_mask.mean(logits_fake)
# [TODO] skip perceptual loss (LPIPS)
d_weight = 1 # [TODO] skip calculate_adaptive_weight
disc_factor = 1 # [TODO] adopt_weight
loss = rec_loss + d_weight * disc_factor * g_loss + qloss

In [56]:
##### Discriminator loss
# [TODO] mask the shorter frames for dis loss
logits_real = discriminator(motions.detach(), src_mask=src_mask, length=length) # [TODO] Can we use the same logits_real from GAN loss??? 
logits_fake = discriminator(recon.detach(), src_mask=src_mask, length=length)
d_loss = disc_factor * vanilla_d_loss(logits_real, logits_fake)

In [9]:
lr = 4.5e-6
opt_ae = torch.optim.Adam(list(encoder.parameters())+
                            list(decoder.parameters())+
                            list(quantize.parameters()),
                            lr=lr, betas=(0.5, 0.9))
opt_disc = torch.optim.Adam(discriminator.parameters(),
                            lr=lr, betas=(0.5, 0.9))

In [10]:

loss.backward()
d_loss.backward()
opt_ae.step()
opt_disc.step()
opt_ae.zero_grad()
opt_disc.zero_grad()